In [1]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm # to create loadbard in for loop
import os.path

In [2]:
# need to precise the option for date, nbr of metada for call transcript in the url paramater 
# https://en.wikipedia.org/wiki/Query_string
nbr_earning_call_metada = 50 # I think 50 is max but can easyily be adapted, with for loop or something like that
date_of_request= "2022-04-28"
url = "https://seekingalpha.com/api/v3/articles"+"?cacheBuster="\
      +str(date_of_request)\
      +"&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0"\
      +"&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]="\
      +str(nbr_earning_call_metada)\
      +"&page[number]=1"
url

'https://seekingalpha.com/api/v3/articles?cacheBuster=2022-04-28&filter[category]=earnings%3A%3Aearnings-call-transcripts&filter[since]=0&filter[until]=0&include=author%2CprimaryTickers%2CsecondaryTickers&isMounting=true&page[size]=50&page[number]=1'

In [3]:
# the chrome driver depend on which version of chrome you are running specify the good one
executable_path = "./chromedriver_linux"
service = Service(executable_path)
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=option)
driver.get(url)
api_json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
#a quick loock of the result to check if we get results are get captcha/bot error 

print(api_json_response[0:100])
api_data_response = json.loads(api_json_response)

{"data":[{"id":"4505479","type":"transcript","attributes":{"publishOn":"2022-04-30T05:14:02-04:00","


## API data response

<pre>
dict( 
    data : list(one element is one earning call metada)
    included : "don't care for now"
    meta : "don't care for now"
        )
</pre>
One earning call metadata is show below 

In [4]:
api_data_response["data"][0]

{'id': '4505479',
 'type': 'transcript',
 'attributes': {'publishOn': '2022-04-30T05:14:02-04:00',
  'isLockedPro': False,
  'commentCount': 0,
  'gettyImageUrl': None,
  'themes': {},
  'title': 'Banco Santander-Chile (BSAC) Management On Q1 2022 Results - Earnings Call Transcript'},
 'relationships': {'author': {'data': {'id': '44211', 'type': 'author'}},
  'sentiments': {'data': []},
  'primaryTickers': {'data': [{'id': '3524', 'type': 'tag'}]},
  'secondaryTickers': {'data': []},
  'otherTags': {'data': [{'id': '96991', 'type': 'tag'},
    {'id': '17862', 'type': 'tag'},
    {'id': '49', 'type': 'tag'},
    {'id': '586376', 'type': 'tag'},
    {'id': '18044', 'type': 'tag'},
    {'id': '17876', 'type': 'tag'}]}},
 'links': {'self': '/article/4505479-banco-santander-chile-bsac-management-on-q1-2022-results-earnings-call-transcript'}}

In [5]:

id_and_path_to_earning_calls = [ (earning_call_metada["id"] ,earning_call_metada["links"]["self"]) 
                     for earning_call_metada in api_data_response["data"]]
# ""check url
id_and_path_to_earning_calls[0:2]

[('4505479',
  '/article/4505479-banco-santander-chile-bsac-management-on-q1-2022-results-earnings-call-transcript'),
 ('4505475',
  '/article/4505475-loyalty-ventures-lylt-ceo-charles-horn-on-q1-2022-results-earnings-call-transcript')]

## download all not yet download html from url

In [6]:
domain_name = 'https://seekingalpha.com'
need_to_download = []
for id_earning_call, path_to_earning_call in tqdm(id_and_path_to_earning_calls):
    if not os.path.isfile("data/article/"+ id_earning_call +".html"):
        # not in local directory
        need_to_download.append((id_earning_call, path_to_earning_call))
print(f"Need to download {len(need_to_download)} html file(s)")
for id_earning_call, path_to_earning_call in tqdm(need_to_download):
    with open("data/article/"+ id_earning_call+".html", "x") as file:
            url = domain_name + path_to_earning_call
            html_transcript = requests.get(url).text
            file.write(html_transcript)
    

100%|████████████████████████████████████████| 50/50 [00:00<00:00, 39038.57it/s]


Need to download 50 html file(s)


100%|███████████████████████████████████████████| 50/50 [00:33<00:00,  1.49it/s]


## Check last file got downloaded, i.e. captcha/bot error 

In [7]:
BeautifulSoup(html_transcript).get_text()[:2000]

"AB Electrolux (publ) (ELUXY) CEO Jonas Samuelson on Q1 2022 Results - Earnings Call Transcript | Seeking Alpha                    Seeking Alpha - Go to HomepageTrendingMy PortfolioMy AuthorsTop StocksLatest NewsMarketsStock IdeasDividendsETFsEducationWatch & ListenTranscriptsConsumerAB Electrolux (publ) (ELUXY) CEO Jonas Samuelson on Q1 2022 Results - Earnings Call TranscriptApr. 29, 2022 5:05 PM ETAB Electrolux (publ) (ELUXY), ELUXFSA Transcripts126.09K FollowersFollowAB Electrolux (publ) (OTCPK:ELUXY) Q1 2022 Results Conference Call April 29, 2022 3:00 AM ET Company Participants Jonas Samuelson - President, Chief Executive Officer Therese Friberg - Chief Financial Officer Sophie Arnius - Head of Investor Relations Conference Call Participants Uma Samlin - Bank of America Andre Kukhnin - Credit Suisse James Moore - Redburn Andreas Willi - JPMorgan Bjorn Enarson - Danske Bank William Turner - Goldman Sachs Karri Rinta - Handelsbanken Olof Cederholm - ABG Sundall Collier Gustav Hageus 